## **Resources**
1. https://www.datacamp.com/community/tutorials/recommender-systems-python/
2. https://blog.codecentric.de/en/2019/07/recommender-system-movie-lens-dataset/
3. http://surpriselib.com/
4. https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
5. https://github.com/NicolasHug/Surprise/blob/master/examples/benchmark.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 1.   Convert Json-File into CSV File

---








In [ ]:
import pandas as pd
from ast import literal_eval

with open("/content/drive/My Drive/Web Mining Project/Dataset/steam_reviews.json") as f:
	lines = f.readlines()
	print("done reading")
	dicts = []
	count = 1
	for line in lines:
		if (count%100000 == 0):
			print(count)
		dicts.append(literal_eval(line))
		count=count+1
	df = pd.DataFrame(dicts)
	df.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/steam_reviews.csv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-109a99fa70f3>", line 5, in <module>
    lines = f.readlines()
  File "/usr/lib/python3.6/codecs.py", line 318, in decode
    def decode(self, input, final=False):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_contex

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
from ast import literal_eval

with open("/content/drive/My Drive/Web Mining Project/Dataset/steam_games.json") as f:
	lines = f.readlines()
	print("done reading")
	dicts = []
	count = 1
	for line in lines:
		if (count%100000 == 0):
			print(count)
		dicts.append(literal_eval(line))
		count=count+1
	df = pd.DataFrame(dicts)
	df.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/steam_games.csv")

done reading


# 2.   Preprocessing Steps





**`Steam_Games`** Rename the column and drop unnecessary columns

In [ ]:
import pandas as pd
import numpy as np
df_games = pd.read_csv('/content/drive/My Drive/Web Mining Project/Dataset/steam_games.csv')
#df.dtypes
df_games.rename(columns={'id':'product_id'}, inplace=True)
#drop unnecessary attributes
df_games.drop(['app_name','url', 'reviews_url', 'release_date'], axis=1, inplace=True)
df_games.head()

,Unnamed: 0,publisher,genres,title,tags,discount_price,specs,price,early_access,product_id,developer,sentiment,metascore
0,0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.49,['Single-player'],4.99,False,761140.0,Kotoshiro,NaN,NaN
1,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive,NaN
2,2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...",NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com,Mostly Positive,NaN
3,3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,"['Action', 'Adventure', 'Casual']",0.83,['Single-player'],0.99,False,767400.0,彼岸领域,NaN,NaN
4,4,NaN,NaN,NaN,"['Action', 'Indie', 'Casual', 'Sports']",1.79,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN,NaN,NaN


Steam_Reviews Rename the column and drop the unnecessary columns

In [ ]:
df_reviews = pd.read_csv('/content/drive/My Drive/Web Mining Project/Dataset/steam_reviews.csv', keep_default_na=False)
df_reviews.head()
# put text column to another csv
df_text = df_reviews[['user_id','text','date','product_id', 'username']].copy()
df_text.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/user_text.csv", index=False)

df_reviews.drop(['page_order', 'early_access', 'username', 'compensation', 'text', 'page'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#replace empty places with nan and delete rows where user_id is missing
df_reviews.replace('', np.nan, inplace=True)
df_reviews = df_reviews.dropna(axis=0, subset=['user_id'])
#df2['user_id'] = df2['user_id'].astype(str)
df_reviews.head()

,Unnamed: 0,hours,products,product_id,date,found_funny,user_id
4,4,16.6,577.0,35140,2018-01-04,NaN,76561198007483075
8,8,1.1,729.0,707610,2017-10-16,NaN,76561197970402776
11,11,3.8,431.0,328100,2017-06-23,2.0,76561198060686749
13,13,14.8,147.0,35140,2018-01-03,NaN,76561198023491401
16,16,17.0,87.0,35140,2018-01-03,NaN,76561198115331805


In [ ]:
#inner join games and reviews on product_id
merged = df_games.merge(df_reviews, how='inner', on='product_id')
merged = merged.dropna(axis=0, subset=['user_id', 'title'])



In [ ]:
merged.dtypes
merged.head()
merged = merged.loc[:, ~merged.columns.str.contains('^Unnamed')]

print("DF size before dropping: " + str(merged.shape))
merged.drop_duplicates(subset=['user_id','title','hours','date'], keep='first', inplace=True)
print("DF size after dropping: " + str(merged.shape))
#merged.to_csv('/content/drive/My Drive/Web Mining Project/Dataset/final_merged.csv', index=False)
#merged.head()

In [ ]:
df_usergamehours = merged[['user_id','product_id','title', 'hours']].copy()
df_usergamehours['count']=1
df_usergamehours['hours'] = df_usergamehours['hours'].astype(float)
df_usergamehours.dtypes

#df_grouped = df_usergamehours.groupby(['user_id', 'title']).sum()
#print(df_grouped.shape)
df_grouped = df_usergamehours.groupby(['user_id', 'title', 'product_id']).sum()
print(df_grouped.shape)
df_grouped = df_grouped.rename(columns={'hours':'total_hours'})
print(df_grouped.head())

(2832522, 2)
                                                                       total_hours  count
user_id           title                                    product_id                    
76561197960265806 Terraria                                 105600.0          280.7      1
76561197960266010 Animation Throwdown: The Quest for Cards 591960.0           28.4      1
76561197960266146 South Park™: The Fractured But Whole™    488790.0            2.1      1
76561197960266225 Paladins®                                444090.0           59.2      1
76561197960266375 Fallout 4                                377160.0         1487.0      1


In [ ]:
df_grouped.head()
df_grouped.shape

(2832522, 2)

In [ ]:
df_grouped=df_grouped.reset_index()
df_grouped.dtypes
df_grouped.head()
#df5000=df_grouped.head(5000).copy()

,user_id,title,product_id,total_hours,count
0,76561197960265806,Terraria,105600.0,280.7,1
1,76561197960266010,Animation Throwdown: The Quest for Cards,591960.0,28.4,1
2,76561197960266146,South Park™: The Fractured But Whole™,488790.0,2.1,1
3,76561197960266225,Paladins®,444090.0,59.2,1
4,76561197960266375,Fallout 4,377160.0,1487.0,1


# 3. Creating Binnings for the individual users



In [ ]:
import numpy as np
#df_grouped.dtypes
df_grouped['user_id'] = df_grouped['user_id'].astype('int64')
df_grouped.dtypes
df_grouped['product_id'] = df_grouped['product_id'].astype('int64')
#df_grouped.drop(['level_0', 'index'], axis=1, inplace=True)
df_grouped.head()
df_grouped.to_csv('/content/drive/My Drive/Web Mining Project/Dataset/user_game_totalhours.csv', index=False)

In [ ]:
df_grouped.sort_values(['user_id'], inplace = True)
df_grouped.head()

,user_id,title,product_id,total_hours,count
0,76561197960265806,Terraria,105600,280.7,1
1,76561197960266010,Animation Throwdown: The Quest for Cards,591960,28.4,1
2,76561197960266146,South Park™: The Fractured But Whole™,488790,2.1,1
3,76561197960266225,Paladins®,444090,59.2,1
4,76561197960266375,Fallout 4,377160,1487.0,1


In [ ]:
df_grouped.shape

(2832522, 5)

In [ ]:
import pandas as pd
import numpy as np
df_grouped = pd.read_csv('/content/drive/My Drive/Web Mining Project/Dataset/filter_user_id.csv')
df_grouped = df_grouped[df_grouped['total_hours'] > 1]
#df_grouped['binrange'] = df_grouped.groupby('user_id').total_hours.apply(pd.cut, bins=5)
print(df_grouped.groupby('user_id').count())
#df_grouped1000['binbin'] = df_grouped1000.groupby('user_id').total_hours.apply(pd.qcut, q=[0, .2, .4, .6, .8, 1], duplicates = 'drop', labels=False)
#print(df_grouped1000['binbin'].value_counts())
#df_grouped1000.to_csv('/content/drive/My Drive/Web Mining Project/Dataset/user_binned_qcuttest.csv', index=False)

                   Unnamed: 0  Unnamed: 0.1  ...  count  bin
user_id                                      ...            
76561197960267615          17            17  ...     17   17
76561197960273775           9             9  ...      9    9
76561197960275370          10            10  ...     10   10
76561197960277861          45            45  ...     45   45
76561197960281570          44            44  ...     44   44
...                       ...           ...  ...    ...  ...
76561198431916153          10            10  ...     10   10
76561198445205155           9             9  ...      9    9
76561198445401211           7             7  ...      7    7
76561198452571798          11            11  ...     11   11
76561198726325843           8             8  ...      8    8

[25428 rows x 13 columns]


# 4. Join the Bin Ouput with our Final-Output

In [ ]:
import pandas as pd
final_file = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/steam_games.csv")
final_file = final_file.drop_duplicates(subset=["id"], keep='first')
bins = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/user_binned_with_9_bins.csv")
final_file.rename(columns={'id':'product_id'}, inplace=True)
#bins = bins[['user_id', 'product_id', 'total_hours', 'bin']].copy()
merge_final_bin = final_file.merge(bins, how='inner', on='product_id')
#print(list(final_file))
#print(bins.head(5))
#print(merge_final_bin.head(5))
#merge_final_bin.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/final_with_bins.csv", index = False)

merge_final_bin.drop(['early_access', 'price','release_date', 'app_name', 'title_y','url', 'discount_price', 'reviews_url', 'metascore', 'developer' ], axis=1, inplace=True)
merge_final_bin.head()
merge_final_bin.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/final_with_bins.csv", index = False)



# Resampling


In [ ]:
from sklearn.utils import resample



df_majority = train[train.bin==1.0]
df_majority_downsampled = resample(df_majority, 
                                replace=False,    # sample without replacement
                                n_samples=38858,     # to match minority class
                                random_state=123)
df_minority1 = train[train.bin==1.5]
df_minority1 = resample(df_minority1, 
                                 replace=True,    # sample without replacement
                                 n_samples=200000,     # to match minority class
                                 random_state=123)
df_minority2 = train[train.bin==2.0]
df_minority2 = resample(df_minority2, 
                                 replace=True,    # sample without replacement
                                 n_samples=195000,     # to match minority class
                                 random_state=123)
df_minority3 = train[train.bin==2.5]
df_minority3 = resample(df_minority3, 
                                 replace=True,    # sample without replacement
                                 n_samples=190000,     # to match minority class
                                 random_state=123)
df_minority4 = train[train.bin==3.0]
df_minority4 = resample(df_minority4, 
                                 replace=True,    # sample without replacement
                                 n_samples=185000,     # to match minority class
                                 random_state=123)
df_minority5 = train[train.bin==3.5]
df_minority5 = resample(df_minority5, 
                                 replace=True,    # sample without replacement
                                 n_samples=180000,     # to match minority class
                                 random_state=123)
df_minority6 = train[train.bin==4.0]
df_minority6 = resample(df_minority6, 
                                 replace=True,    # sample without replacement
                                 n_samples=175000,     # to match minority class
                                 random_state=123)
df_minority7 = train[train.bin==4.5]
df_minority7 = resample(df_minority7, 
                                 replace=True,    # sample without replacement
                                 n_samples=170000,     # to match minority class
                                 random_state=123)
df_minority8 = train[train.bin==5.0]
df_minority8 = resample(df_minority8, 
                                 replace=True,    # sample without replacement
                                 n_samples=220000,     # to match minority class
                                 random_state=123)

trainset = pd.concat([df_majority, df_minority1, df_minority2, df_minority3, df_minority4, df_minority5, df_minority6, df_minority7, df_minority8])

# 5. Creating Train and Test set

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Web Mining Project/Dataset/"
%cd /content/drive/My Drive/Web Mining Project/Dataset/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Archiv			     steam_games.csv
 cb5000.csv		     steam_games.json
 content_based_result.csv    steam_reviews.csv
'Data for Algorithms'	     steam_reviews.json
 filter_for_algorithms.csv   try.csv
 filter_user_id.csv	     user_binned_qcut.csv
 final_with_bins.csv	     user_binned_qcuttest.csv
 predicitons.csv	     user_binned_with_9_bins.csv
 predicitons_game.csv	     user_binned_with_9_bins_new.csv
 predicitons_item.csv	     user_game_totalhours.csv
 predicitons_user.csv	     user_text.csv
/content/drive/.shortcut-targets-by-id/1-qkg6a-0KpYJwBNKZH3n-imXaz3kcFRX/Web Mining Project/Dataset


In [ ]:
ratings = pd.read_csv("final_with_bins.csv")

In [ ]:
#Filtering out User that have 
ratings = ratings.groupby('product_id').filter(lambda x: len(x) >= 5)
ratings = ratings.groupby('user_id').filter(lambda x: len(x) >= 15)

# Make correct types
ratings.product_id = ratings.product_id.astype('category')
ratings.head()
# Create user & product_id mappings.
userIds = ratings['user_id'].unique()
print("There are %i unique users.", len(userIds))
productIds = ratings['product_id'].unique()
print("There are %i unique products.", len(productIds))



In [ ]:

#ratings.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/filter_for_algorithms.csv", index = False)
ratings = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/filter_for_algorithms.csv")

ratings.shape

(308334, 14)

In [ ]:
!pip install scikit-surprise
from surprise import Dataset, Reader, SVD, accuracy, NMF
from surprise.model_selection import train_test_split 

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(ratings[['user_id','product_id','bin']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

     |████████████████████████████████| 6.5MB 38kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675388 sha256=e2783634dce43b29290c6adf6833c0c2c4b2208a9f48641fd4332a70b0d785f2
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


## 5.1. Functions


In [ ]:
def opt_parameters(exp_function, trainset, testset, config=None, param_space_iter=None):
    runs_info = collections.OrderedDict()
    for params in param_space_iter:
        # get key/id
        param_id = ';'.join(['%s=%s' % (key, value) for (key, value) in params.items()])
        print(param_id)
        # run experiment
        exp_info = run_exp(exp_function, trainset, testset, config, config_updates=params)
        # add results to OrderedDict
        runs_info[param_id] = exp_info
    
    return runs_info

In [ ]:
def run_exp(exp_function, trainset, testset, config=None, config_updates=None):  
    if config is None:
        config = {}
    else:
        config = config.copy()
    
    if config_updates is not None:
        config.update(config_updates)
        
    return exp_function(config, trainset, testset)

In [ ]:
def evaluation(predictions):
    
       # Get evaluation metrics (RMSE, MAE, MAP, NDCG) for predictions
  
    res = {}
    
    #res['rmse'] = accuracy.rmse(predictions)
    #res['mae'] = accuracy.mae(predictions)
    res['ndcg'] = ndcg(predictions)
    res['map'] = mean_ap(predictions)
    
    return res

Evaluation Functions

In [ ]:
from collections import defaultdict
from math import log2

import numpy as np


def mean_ap(predictions, verbose=True, k=5):
    predictions_u = defaultdict(list)
    true_u = defaultdict(list)

    #for u0, i0, r0, est, _ in predictions: # for algorithms
    #    true_u[u0].append((r0, i0))
    #    predictions_u[u0].append((est, i0))
        
    for index, (u0, i0, r0, est) in predictions.iterrows(): #for hybrid method
       true_u[u0].append((r0, i0))
       predictions_u[u0].append((est, i0))
       
    aps = list()

    for u0 in predictions_u:
        if len(predictions_u[u0]) >= k:
            p = sorted(predictions_u[u0], reverse=True)[:k]
            p = [v for _,v in p]
            t = sorted(true_u[u0], reverse=True)[:k]
            t = [v for _,v in t]

            ap = 0
            correct = 0
            for i in range(len(p)):
                if p[i] in t:
                    correct += 1
                    ap += correct / (i+1)

            ap /= len(t)
            aps.append(ap)
            
    map_ = np.array(aps).mean()
    
    if verbose:
        print('MAP: {0:1.4f}'.format(map_))
    
    return map_


def ndcg(predictions, verbose=True, k=5):
    predictions_u = defaultdict(list)
    true_u = defaultdict(list)

    #for u0, i0, r0, est, _ in predictions:
    #    true_u[u0].append((r0, i0))
    #    predictions_u[u0].append((est, i0))

    for index,(u0, i0, r0, est) in predictions.iterrows(): #eval for hybrid
        true_u[u0].append((r0, i0))
        predictions_u[u0].append((est, i0))

    idcg = np.arange(k+1)
    idcg[0:2] = 2
    idcg = 1 / np.log2(idcg)
    idcg = np.cumsum(idcg)

    ndcgs = list()

    for u0 in predictions_u:
        if len(predictions_u[u0]) >= k:
            p = sorted(predictions_u[u0], reverse=True)[:k]
            p = [v for _,v in p]
            t = sorted(true_u[u0], reverse=True)[:k]
            t = [v for _,v in t]

            dcg = 0
            correct = 0
            for i in range(len(p)):
                if p[i] in t:
                    correct += 1
                    if i == 0:
                        dcg += 1
                    else:
                        dcg += 1/log2(i+1)

            ndcg = dcg / idcg[correct]
            ndcgs.append(ndcg)

    ndcg_ = np.array(ndcgs).mean()
    
    if verbose:
        print('NDCG: {0:1.4f}'.format(ndcg_))
        
    return ndcg_

## 5.2. Baseline

In [ ]:
import collections
from sklearn.model_selection import ParameterGrid
from surprise import BaselineOnly

In [ ]:
def train_BaselineOnly(config, trainset, testset):
    predictions = BaselineOnly(
        bsl_options = {
            'method': config['method'],
            #'reg': config['reg']
        }).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    
    return info

In [ ]:
config = {
    'method': 'als',
    #'reg': 0.02
}

param_grid = {
    'method': np.array(['als']),
    #'reg': np.array([0.1, 0.02, 0.03])
}

baselineOnly_runs_info = opt_parameters(train_BaselineOnly, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

method=als
Estimating biases using als...
RMSE: 0.8519
MAE:  0.5238
NDCG: 0.7384
MAP: 0.6906


## 5.3. SVD

In [ ]:
from sklearn.model_selection import ParameterGrid
import collections


In [ ]:
def train_SVD(config, trainset, testset):
    predictions = SVD(
        n_epochs = config['epochs'],
        lr_all = config['learning_rate'],
        reg_all = config['regularization']
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    
    return info

In [ ]:
svd_runs_info = opt_parameters(train_SVD, config={
    'epochs': 20,
    'learning_rate': 0.005,
    'regularization': 0.02
}, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid = {
    'epochs': np.array([20, 35, 50]), # number of iterations
    'learning_rate': np.array([0.01, 0.002, 0.005]), # learning rate
    'regularization': np.array([0.02, 0.05]), # regularization term
}))

epochs=20;learning_rate=0.01;regularization=0.02
RMSE: 0.8600
MAE:  0.5139
NDCG: 0.7358
MAP: 0.6839
epochs=20;learning_rate=0.01;regularization=0.05
RMSE: 0.8482
MAE:  0.5067
NDCG: 0.7375
MAP: 0.6890
epochs=20;learning_rate=0.002;regularization=0.02
RMSE: 0.8541
MAE:  0.5301
NDCG: 0.7372
MAP: 0.6829
epochs=20;learning_rate=0.002;regularization=0.05
RMSE: 0.8526
MAE:  0.5291
NDCG: 0.7368
MAP: 0.6840
epochs=20;learning_rate=0.005;regularization=0.02
RMSE: 0.8517
MAE:  0.5145
NDCG: 0.7395
MAP: 0.6910
epochs=20;learning_rate=0.005;regularization=0.05
RMSE: 0.8448
MAE:  0.5120
NDCG: 0.7383
MAP: 0.6922
epochs=35;learning_rate=0.01;regularization=0.02
RMSE: 0.8623
MAE:  0.5157
NDCG: 0.7363
MAP: 0.6868
epochs=35;learning_rate=0.01;regularization=0.05
RMSE: 0.8489
MAE:  0.5047
NDCG: 0.7376
MAP: 0.6866
epochs=35;learning_rate=0.002;regularization=0.02
RMSE: 0.8493
MAE:  0.5175
NDCG: 0.7391
MAP: 0.6910
epochs=35;learning_rate=0.002;regularization=0.05
RMSE: 0.8471
MAE:  0.5169
NDCG: 0.7402
MAP: 0

## 5.4. NMF

In [ ]:
from sklearn.model_selection import ParameterGrid
import collections

In [ ]:
def train_NMF(config, trainset, testset):
    predictions = NMF(
        n_epochs = config['epochs'],
        n_factors = config['n_factors'],
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
   
    return info

In [ ]:
NMF_runs_info = opt_parameters(train_NMF, config={
    'n_factors': 5,
    'epochs': 20,
   
}, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid = {
    'n_factors': np.array([10, 15, 20]), #number of factors 
    'epochs': np.array([20, 35, 50]), # number of iterations of NMF procedure
    
    
}))

epochs=20;n_factors=10
RMSE: 0.8998
MAE:  0.5280
NDCG: 0.7360
MAP: 0.6910
epochs=20;n_factors=15
RMSE: 0.9000
MAE:  0.5450
NDCG: 0.7375
MAP: 0.6889
epochs=20;n_factors=20
RMSE: 0.9003
MAE:  0.5555
NDCG: 0.7366
MAP: 0.6865
epochs=35;n_factors=10
RMSE: 0.9033
MAE:  0.4893
NDCG: 0.7411
MAP: 0.7197
epochs=35;n_factors=15
RMSE: 0.8981
MAE:  0.4816
NDCG: 0.7414
MAP: 0.7153
epochs=35;n_factors=20
RMSE: 0.8849
MAE:  0.4733
NDCG: 0.7405
MAP: 0.7150
epochs=50;n_factors=10
RMSE: 0.9069
MAE:  0.4927
NDCG: 0.7410
MAP: 0.7101
epochs=50;n_factors=15
RMSE: 0.9060
MAE:  0.4924
NDCG: 0.7410
MAP: 0.7092
epochs=50;n_factors=20
RMSE: 0.8964
MAE:  0.4833
NDCG: 0.7390
MAP: 0.7086


## 5.5. KNN

In [ ]:
from surprise import KNNBasic
import pandas as pd

def train_KNNBasic(config, trainset, testset):
    predictions = KNNBasic(
        k = config['k'],
        sim_options = {
            'name': config['sim_name'],
            'min_support': config['min_support'],
            'user_based': config['user_based']
        }
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    print(predictions[0:1000])
   
    return info

In [ ]:
from surprise import KNNWithMeans
def train_KNNWithMeans(config, trainset, testset):
    predictions = KNNWithMeans(
        k = config['k'],
        sim_options = {
            'name': config['sim_name'],
            'min_support': config['min_support'],
            'user_based': config['user_based']
        }
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    return info

### 5.5.1. Item-Based CF

Knn Baseline

In [ ]:
config = {
    'k': 10,
    'sim_name': 'cosine',
    'min_support': 5,
    'user_based': False
}

param_grid = {
    'k': np.array([5, 10, 20]),
    'sim_name': np.array(['cosine', 'msd', 'pearson', 'pearson_baseline']),
    'min_support': np.array([1, 3, 5])}



ib_knnbasic_runs_info = opt_parameters(train_KNNBasic, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid), )

Output hidden; open in https://colab.research.google.com to view.

KNN with Means

In [ ]:
config = {
    'k': 30,
    'sim_name': 'cosine',
    'min_support': 1,
    'user_based': False
}

param_grid = {
    'k': np.array([5, 10, 20, 30]),
    'sim_name': np.array(['cosine', 'pearson_baseline']),
    'min_support': np.array([2, 3, 5])
}


ib_knnmeans_runs_info = opt_parameters(train_KNNWithMeans, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

k=5;min_support=2;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9052
MAE:  0.5378
NDCG: 0.7322
MAP: 0.6827
k=5;min_support=2;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9361
MAE:  0.5314
NDCG: 0.7410
MAP: 0.7077
k=5;min_support=3;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9182
MAE:  0.5416
NDCG: 0.7317
MAP: 0.6808
k=5;min_support=3;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9401
MAE:  0.5315
NDCG: 0.7407
MAP: 0.7051
k=5;min_support=5;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9291
MAE:  0.5453
NDCG: 0.7330
MAP: 0.6849
k=5;min_support=5;sim_name=pearson_baseline
Estimating biases using als...
Computing the pear

### 5.5.2. User-Based CF

KNN Baseline

In [ ]:
config = {
    'k': 5,
    'sim_name': 'cosine',
    'min_support': 1,
    'user_based': True
}

param_grid = {
    'k': np.array([5, 10, 20]),
    'sim_name': np.array(['cosine', 'msd', 'pearson', 'pearson_baseline']),
    'min_support': np.array([2, 3, 5])
}


ib_knnbasic_runs_info = opt_parameters(train_KNNBasic, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

Output hidden; open in https://colab.research.google.com to view.

KNN with means

In [ ]:
config = {
    'k': 5,
    'sim_name': 'cosine',
    'min_support': 1,
    'user_based': True
}

param_grid = {
    'k': np.array([5,10,20]),
    'sim_name': np.array(['cosine','pearson_baseline']),
    'min_support': np.array([2, 3, 5])
}


ib_knnmeans_runs_info = opt_parameters(train_KNNWithMeans, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

k=5;min_support=2;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9142
MAE:  0.5533
NDCG: 0.7378
MAP: 0.7019
k=5;min_support=2;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9160
MAE:  0.5536
NDCG: 0.7379
MAP: 0.6900
k=5;min_support=3;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9308
MAE:  0.5687
NDCG: 0.7401
MAP: 0.7020
k=5;min_support=3;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9433
MAE:  0.5742
NDCG: 0.7386
MAP: 0.6906
k=5;min_support=5;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9637
MAE:  0.5939
NDCG: 0.7426
MAP: 0.6986
k=5;min_support=5;sim_name=pearson_baseline
Estimating biases using als...
Computing the pear

## 5.6. Hybrid

In [ ]:
from surprise import KNNBasic
import pandas as pd

def train_KNNBasic(config, trainset, testset):
    predictions = KNNBasic(
        k = config['k'],
        sim_options = {
            'name': config['sim_name'],
            'min_support': config['min_support'],
            'user_based': config['user_based']
        }
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    predictions = pd.DataFrame(predictions)
    #print(predictions[0:1000])
    predictions.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/predicitons_game.csv")
   
    return info

In [ ]:
config = {
    'k': 5, #for item based
    #'k' : 10, #for user based
    'sim_name': 'pearson_baseline',
    'min_support': 5,
    'user_based': False #item-based
    #'user_based': True #user-based
}

param_grid = {}
ib_knnbasic_runs_info = opt_parameters(train_KNNBasic, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid), )


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
NDCG: 0.7620
MAP: 0.7443


In [ ]:

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


def evaluation(predictions):
   
    res = {}
    
    #res['rmse'] = accuracy.rmse(predictions)
    y_true = user_pred['r_ui']
    y_pred = user_pred['est']
    res['rmse'] = mean_squared_error(y_true, y_pred, squared=False)
    
    res['mae'] = mean_absolute_error(y_true, y_pred)

    res['ndcg'] = ndcg(predictions)
    res['map'] = mean_ap(predictions)
    
    return res


In [ ]:
user_pred = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/predicitons_user.csv")
user_pred = user_pred[['uid', 'r_ui','est', 'iid']].copy()


game_pred = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/predicitons_game.csv")
game_pred = game_pred[['uid', 'r_ui','est', 'iid']].copy()


#merge = test1.merge(test2, how='inner', on='uid')
user_pred['est2']= game_pred['est']

#user_pred['hybrid'] = (user_pred.est*0.6 + user_pred.est2*0.4) #put more weight on user-based since it performs slightly better

user_pred['hybrid'] = (user_pred.est + user_pred.est2)/2

user_pred = user_pred.drop(columns = ['est', 'est2'], axis = 1)

user_pred = user_pred.rename({'hybrid': 'est'}, axis=1)


print(evaluation(user_pred))


NDCG: 0.8102
MAP: 0.9171
{'rmse': 0.9667065796910703, 'mae': 0.6129372542568645, 'ndcg': 0.810159797084038, 'map': 0.9170620021512877}


In [ ]:
user_pred = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/try.csv")
print(user_pred.head())
user_pred = user_pred[['uid', 'r_ui','est', 'iid']].copy()

print(evaluation(user_pred))

   uid  iid  r_ui  est
0    1    1     1    0
1    1    2     1    1
2    1    3     1    1
3    1    4     0    1
4    1    5     0    0
NDCG: 0.7808
MAP: 1.0000
{'ndcg': 0.7807789774739649, 'map': 1.0}


## 5.7. Naïve Bayes


In [ ]:


df = ratings[['user_id', 'product_id', 'bin']].copy()
df_unique_1000 = df['user_id'].unique()[0:1000]
df_nb = df[df['user_id'].isin(df_unique_1000)]

print(df_nb.shape)
print(df.shape)

(32540, 3)
(475665, 3)


In [ ]:
# naive bayes for recommendation system

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split




df_nb['product_id'] = df_nb['product_id'].astype(int)
df_nb['bin'] = df_nb['bin'].astype(str)
df_matrix = df_nb.pivot(values='bin', index='user_id', columns='product_id')

# print(df_matrix.head(10))
# print(df['user_id'].nunique())
# print(df['product_id'].nunique())
print("Matrix shape: " + str(df_matrix.shape))

count=0

# array storing all recommendations
arr_rec = []

for user in df_matrix.index.tolist():
	print(count)
	
	# prods user has not rated
	prods_unrated = df_matrix.columns[df_matrix.loc[user].isnull()]
	prods_unrated = prods_unrated.tolist()
	#print(prods_unrated)
	
	# prods user has rated
	prods_rated = set(list(df_matrix)) - set(prods_unrated)
	prods_rated = list(prods_rated)
	#print(prods_rated)

	# predictions for all products for current user
	p_user = []

	if len(prods_unrated) == 0:
		# user rated all products, no need to recommend
		continue

	for p in prods_unrated:
		# probability of different ratings for p
		p_pr = []

		for r in range(1,5):
			p_Y = len(df_matrix[df_matrix[p]==str(r)]) / (len(df_matrix) - 1)
			p_XY = 1

			# users who give rating r to unrated item p
			df_matrix_Y = df_matrix[df_matrix[p]==str(r)].copy()

			# if nobody rated p before
			if len(df_matrix_Y) == 0:
				p_pr.append(0.0)
				continue
			
			# print("p unrated: " + str(p))
			# print("r: " + str(r))
			# print(df_matrix_Y.head(10))

			for q in prods_rated:
				
				# print("q: " + str(q))

				# rating of user on item q
				user_r_q = df_matrix.at[user, q]

				# users who give the same rating to item q as user
				df_matrix_r_q = df_matrix_Y[df_matrix[q]==str(user_r_q)]
				
				p_XY = p_XY * float(len(df_matrix_r_q) + 1.0) / float(len(df_matrix_Y) + 5.0)

				# print(str(len(df_matrix_r_q)) + "/" + str(len(df_matrix_Y)))
				# print("p_XY: " + str(p_XY))
				# print("p_Y: " + str(p_Y))

			p_YX = p_XY * p_Y
			p_pr.append(p_YX)
			# print("P(item " + str(p) + "=" + str(r) + "|X) = " + str(p_YX))	

		#print(p_pr)
		#print(str(user) + " will most likely give item " + str(p) + " a score of " + str(max(p_pr)))

		p_user.append(max(p_pr))

	#rec_dict = dict(zip(prods_unrated, p_user))
	#rec_dict_sorted = {k: v for k, v in sorted(rec_dict.items(), key=lambda item: item[1], reverse=True)}
	# print(p_user)
	# print(rec_dict_sorted)
	#print("top recommendation for " + str(user) + ": " + str(prods_unrated[p_user.index(max(p_user))]))
	#print('\n\n')
	for i in range(0,len(p_user)):
		arr_rec.append([user,prods_unrated[i],p_user[i]])
	df_rec = pd.DataFrame(arr_rec, columns=['user_id','product_id','rating'])

	count = count+1
	# if count > 100:
	# 	break


df_rec.to_csv("nb_result.csv", index=False)


In [ ]:
#Test1
ratings.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import ParameterGrid
import collections
def convert(ratings):
    ids = []
    length = 10000
    users = ratings['user_id'].values
    product = ratings['product_id'].values
    for i in range(0, length):
        ids.append(str(users[i])+' '+str(product[i]))
    return ids
def predict(train_data, test_data):
    vectorizer = CountVectorizer()
    train_ids = []
    #length = len(train_data['user_id'])
    train_ids = convert(train_data)
    inputs = vectorizer.fit_transform(train_ids)
    classifier = MultinomialNB()
    outputs = train_data['bin'].values
    classifier.fit(inputs, outputs)
    test_ids = convert(test_data)
    test = vectorizer.transform(test_ids)
    predictions = classifier.predict(test)
    
    return predictions

    #convert(trainset)
    #convert(testset)
predictions = predict(trainset, testset)
info = evaluation(predictions)
print(info)

In [ ]:
def trainnb(config, trainset, testset):
    predictions = MultinomialNB().fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    # return metrics
    return info

In [ ]:
svd_runs_info = opt_parameters(trainnb, convert, predict, config={}, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid = {}))

## 5.8. Cross Validation

In [ ]:
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import KNNBasic
reader = Reader(rating_scale=(1, 5))
trainset, testset = train_test_split(ratings[['user_id','product_id','bin']], test_size=0.2, random_state=0)
trainset.index = range(len(trainset)) # 75627
testset.index = range(len(testset)) # 25209


trainset = Dataset.load_from_df(trainset, reader)

#testset = testset.values.tolist()
benchmark = []
i = 0
alg_list = ['KNNBasic(k=10, sim_options={'name': 'pearson_baseline','user_based': True}']
for algorithm in [KNNBasic(k=10, sim_options={'name': 'pearson_baseline','user_based': True, 'min_support': 5})]:
    # Perform cross validation
    results = cross_validate(algorithm, trainset, measures=['RMSE','MAE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([alg_list[i]], index=['Algorithm']))
    benchmark.append(tmp)
    i+=1
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

SyntaxError: ignored

In [ ]:
from surprise.model_selection import KFold
kf = KFold(n_splits=5)

#algo = SVD()
config={
     'k': 5, #for item based
    #'k' : 10, #for user based
    'sim_name': 'pearson_baseline',
    'min_support': 5,
    'user_based': False #item-based
    #'user_based': True #user-based
}
for trainset, testset in kf.split(data):
  
  #trainSVD(config, trainset, testset)
  knn_userbased = opt_parameters(train_KNNBasic, trainset=trainset, testset=testset, config= {'k': 5, 'sim_name': 'pearson_baseline', 'min_support': 5,'user_based': True}, param_space_iter=ParameterGrid(param_grid = {
    'k': np.array([5, 10]),
    'sim_name': np.array(['cosine', 'pearson_baseline']),
    'min_support': np.array([1,5]),
  }))
    

k=5;min_support=1;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
NDCG: 0.7379
MAP: 0.7004
k=5;min_support=1;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
NDCG: 0.7483
MAP: 0.7334
k=5;min_support=5;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
NDCG: 0.7520
MAP: 0.7423
k=5;min_support=5;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
NDCG: 0.7626
MAP: 0.7541
k=10;min_support=1;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
NDCG: 0.7350
MAP: 0.6870
k=10;min_support=1;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
NDCG: 0.7451
MAP: 0.7131
k=10;min_support=5;sim_name=cosine


# 6. Content-Based Recommendation

In [ ]:
ratings[ratings.isna().any(axis=1)]
ratings['specs'].fillna("", inplace=True)
ratings[ratings.isna().any(axis=1)]
ratings.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,publisher,genres,title_x,tags,specs,product_id,sentiment,user_id,total_hours,count,bin
0,0,20,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",643980.0,Mostly Positive,76561198022842797,11.0,1,1.5
1,1,34,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",643980.0,Mostly Positive,76561198065664667,0.4,1,1.0
2,2,35,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",643980.0,Mostly Positive,76561198073092169,0.8,1,1.0
3,3,36,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",643980.0,Mostly Positive,76561198074855431,2.7,1,1.0
4,4,38,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",643980.0,Mostly Positive,76561198092622384,1.6,1,1.0


In [ ]:
games = ratings[['product_id', 'specs', 'genres', 'tags', 'title_x']].copy()
print(games.head())
games_new = games.drop_duplicates(keep='first')
print(games_new.shape, 'games_new')
print(games.shape, 'games')


   product_id  ...    title_x
0    643980.0  ...  Ironbound
1    643980.0  ...  Ironbound
2    643980.0  ...  Ironbound
3    643980.0  ...  Ironbound
4    643980.0  ...  Ironbound

[5 rows x 5 columns]
(7494, 5) games_new
(470658, 5) games


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(games_new['specs'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=games_new.index.tolist())
# 9724 movies, using 1675 terms to describe
print(tfidf_matrix)
print(tfidf_df)

  (0, 32)	0.37023735242824896
  (0, 2)	0.37023735242824896
  (0, 5)	0.1379134431448606
  (0, 43)	0.1379134431448606
  (0, 0)	0.12653269815851154
  (0, 41)	0.2172681421527372
  (0, 25)	0.3165856347369508
  (0, 30)	0.3165856347369508
  (0, 11)	0.3165856347369508
  (0, 27)	0.3039448906230521
  (0, 24)	0.3937880173935774
  (0, 31)	0.26101287936286655
  (0, 37)	0.0912895727814131
  (1, 5)	0.3866572317615634
  (1, 43)	0.3866572317615634
  (1, 41)	0.3045689255488863
  (1, 24)	0.5520164721953326
  (1, 31)	0.48785438696044825
  (1, 37)	0.2559414999397928
  (2, 15)	0.47811453720587205
  (2, 6)	0.47811453720587205
  (2, 1)	0.47811453720587205
  (2, 45)	0.47811453720587205
  (2, 24)	0.20711940441672994
  (2, 31)	0.18304546179118764
  :	:
  (7490, 5)	0.3153052965937292
  (7490, 43)	0.3153052965937292
  (7490, 41)	0.4967303726140798
  (7490, 24)	0.450149908478902
  (7490, 31)	0.3978279973565134
  (7490, 37)	0.2087112406549393
  (7491, 15)	0.4865442266007559
  (7491, 6)	0.4865442266007559
  (7491, 1)

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
ratings_f1 = ratings[['product_id', 'user_id', 'bin']].copy()
ratings_f2 = ratings_f1.pivot(index = 'product_id', columns ='user_id', values = 'bin').fillna(0)
ratings_f2.head()

user_id,76561197960267615,76561197960273775,76561197960275370,76561197960277861,76561197960281570,76561197960282791,76561197960283776,76561197960284046,76561197960286960,76561197960289679,76561197960292106,76561197960298812,76561197960302047,76561197960318168,76561197960320995,76561197960337918,76561197960344050,76561197960344344,76561197960360633,76561197960380360,76561197960385255,76561197960389247,76561197960406672,76561197960409779,76561197960416558,76561197960419710,76561197960426753,76561197960431441,76561197960436562,76561197960440131,76561197960446959,76561197960447880,76561197960448494,76561197960452964,76561197960456205,76561197960458147,76561197960465223,76561197960465381,76561197960467832,76561197960473045,...,76561198388889834,76561198389680135,76561198391273080,76561198391873454,76561198393073913,76561198393236347,76561198393825451,76561198394681654,76561198395351334,76561198397345172,76561198397704388,76561198397718315,76561198397789355,76561198398217033,76561198399895281,76561198400280187,76561198400486602,76561198400978118,76561198404257091,76561198404355237,76561198405310776,76561198406769954,76561198408159536,76561198408671944,76561198409966219,76561198410099861,76561198410426991,76561198417151798,76561198417996924,76561198418549470,76561198419548776,76561198419787884,76561198421569236,76561198422475968,76561198427374354,76561198431916153,76561198445205155,76561198445401211,76561198452571798,76561198726325843
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
game_list_rating = ratings.product_id.unique().tolist()
games_filter = game_list_rating

In [ ]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        mean_item_rating = ratings.mean(axis=0)
        ratings_diff = (ratings - mean_item_rating[np.newaxis, :])
        pred = mean_item_rating[np.newaxis, :] + ratings_diff.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    elif type == 'content':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred.clip()

In [ ]:
def recommend_movies(pred_df, userID, games, original_ratings, num_recommendations, method_name):
    # Get and sort the user's predictions
    #user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = pred_df.loc[userID].sort_values(ascending=False) # User ID starts at 1

    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == int(userID)]
    #user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
    #                 sort_values(['rating'], ascending=False)
    #             )
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (games[~games['product_id'].isin(user_data['product_id'])].
         merge(pd.DataFrame(sorted_user_predictions).rename_axis('product_id').reset_index(), how = 'left', 
               on = 'product_id').
         rename(columns = {userID: method_name}).
         sort_values(method_name, ascending = False).
                       iloc[:num_recommendations, :]
                      )

    return user_data, recommendations

In [ ]:
content_prediction = predict(ratings_f2.T, content_correlation, type='content')
content_pred_df = pd.DataFrame(content_prediction, columns = list(games_filter))
print(content_pred_df.dtypes)
print(content_pred_df.index)
print('Content-based RS with ranking')
user_full, recommendations = recommend_movies(content_pred_df, 76561197960267615, games_filter, ratings_f1, 5, 'CT_3a_pred_rating')
recommendations


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

ds = ratings
ds.specs = ds.specs.astype(str)
print(ds.dtypes)

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['specs'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in ds.iterrows():
     similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
     similar_items = [(cosine_similarities[idx][i], ds['product_id'][i]) for i in similar_indices]

     results[row['product_id']] = similar_items[1:]

print('done!')


def item(id):
    return ds.loc[ds['product_id'] == id]['specs'].tolist()[0].split(' - ')[0]


 # Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")


recommend(item_id=30.0, num=5)

level_0             int64
index               int64
Unnamed: 0          int64
Unnamed: 0.1        int64
Unnamed: 0.1.1      int64
publisher          object
genres             object
title_x            object
tags               object
specs              object
product_id        float64
sentiment          object
user_id             int64
total_hours       float64
count               int64
bin               float64
new_col             int64
dtype: object
done!
Recommending 5 products similar to ['Multi-player', 'Valve Anti-Cheat enabled']...
-------
Recommended: ['Multi-player', 'Valve Anti-Cheat enabled'] (score:1.0000000000000002)
Recommended: ['Multi-player', 'Valve Anti-Cheat enabled'] (score:1.0000000000000002)
Recommended: ['Multi-player', 'Valve Anti-Cheat enabled'] (score:1.0000000000000002)
Recommended: ['Multi-player', 'Valve Anti-Cheat enabled'] (score:1.0000000000000002)
Recommended: ['Single-player', 'Multi-player', 'Valve Anti-Cheat enabled'] (score:0.9742335185574383)


In [ ]:
# content-based recommender system

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from itertools import chain, repeat

chainer = chain.from_iterable

#---------------------- START functions ----------------------
# Function that takes in movie title as input and outputs most similar movies
# def get_recommendations(prod, cosine_sim):

#     # Get the pairwise similarity scores of games
#     sim_scores = list(enumerate(cosine_sim[prod]))

#     # Sort games based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 10 most similar games
#     sim_scores = sim_scores[1:11]

#     # Get the game indices
#     movie_indices = [i[0] for i in sim_scores]

#     # Return the top 10 most similar movies
#     return games_data.iloc[movie_indices]


def create_pivot_table(pd_df):
    data = pd_df.values
    rows, row_pos = np.unique(data[:, 4], return_inverse=True) # col 4: internal user id
    cols, col_pos = np.unique(data[:, 1], return_inverse=True) # col 1: real product id
    pivot_matrix = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_matrix[:] = np.nan
    pivot_matrix[row_pos, col_pos] = data[:, 2]
    return pivot_matrix


def recommend_movies(pred_df, userID, games, original_ratings, num_recommendations):
    # get internal id of user
    uid_internal = map_uid.at[userID, 'index']

    sorted_user_predictions = pred_df.iloc[uid_internal].sort_values(ascending=False) 

    # get user's rated games
    user_data = original_ratings[original_ratings.user_id == int(userID)]
   	
   	# optional: merge game info
    #user_full = (user_data.merge(games, how='left', left_on='product_id', right_on='product_id').
    #                 sort_values(['rating'], ascending=False)
    #             )
    
    # Recommend the highest predicted rating games that the user hasn't seen yet.
    recommendations = games_data[~games_data['product_id'].isin(user_data['product_id'])].merge(pd.DataFrame(sorted_user_predictions).rename_axis('product_id').reset_index(), how='left', on='product_id').rename(columns={uid_internal: 'pred'}).sort_values('pred', ascending=False).iloc[:num_recommendations,:]

    return recommendations


#---------------------- END functions ----------------------

data = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/filter_for_algorithms.csv", usecols=['product_id', 'user_id', 'title_x', 'publisher', 'genres', 'tags', 'specs', 'bin'])
data['product_id'] = data['product_id'].astype(int) # product_id is initially a float with .0s
print("Total users: " + str(data['user_id'].nunique()))
print("Total products: " + str(data['product_id'].nunique()))

# dataframe storing games info
games_data = data[['product_id', 'title_x', 'publisher', 'genres']].copy()
games_data.fillna("", inplace=True)
games_data.drop_duplicates(inplace=True)
games_data.set_index(pd.Series(np.arange(len(games_data))), inplace=True)
games_data['metadata'] = games_data[['publisher', 'genres']].apply(lambda x: ' '.join(x), axis = 1)

# produce tfidf matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(games_data['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=games_data.index.tolist())

# compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#recommendations = get_recommendations(1, cosine_sim)
#print(recommendations)

# create internal product id
map_pid = games_data.reset_index()
map_pid = map_pid[['index', 'product_id']].copy()

# create internal user id
map_uid = data[['user_id']].copy().drop_duplicates(keep='first')
map_uid.set_index(pd.Series(np.arange(len(map_uid))), inplace=True)
map_uid.reset_index(inplace=True)
map_uid.set_index('user_id', inplace=True)

user_item = data[['user_id','product_id','bin']].copy()
user_item = user_item.merge(map_pid, on='product_id', how='left')
user_item = user_item.merge(map_uid, on='user_id', how='left')

#user_item_matrix = user_item[['user_id','index','bin']].pivot(index='user_id', columns='index', values = 'bin').fillna(0)
user_item_matrix2 = create_pivot_table(user_item)

#pred = user_item_matrix2.dot(cosine_sim) / np.array([np.abs(cosine_sim).sum(axis=1)])

mean_item_rating = np.nanmean(user_item_matrix2, axis=0)
ratings_diff = (user_item_matrix2 - mean_item_rating[np.newaxis, :])

array = np.zeros((len(map_uid), len(map_pid)))
index = 0
for r1 in ratings_diff:
	sum_rated = np.nansum(np.where(np.isnan(r1), r1, cosine_sim), axis=1)
	dp = np.where(np.isnan(r1),0.0,r1).dot(np.where(np.isnan(cosine_sim),0.0,cosine_sim))
	dp = (np.nan_to_num(r1)).dot(cosine_sim)
	prod = np.nan_to_num(dp / sum_rated)
	array[index] = prod
	index=index+1

pred = mean_item_rating[np.newaxis, :] + array

# pred = mean_item_rating[np.newaxis, :] + ratings_diff.dot(cosine_sim) / np.array([np.abs(cosine_sim).sum(axis=1)])
#print(pred)
content_prediction = pred.clip(min=0)

content_pred_df = pd.DataFrame(content_prediction, columns=list(map_pid['product_id']))

# dict in the format {uid: [prod1, prod2, ..., prodN]}, stores all user's recommendations
results = []

# iterate over all users
for index, user in enumerate(data['user_id'].unique().tolist()):
	if index%1000==0:
		print(index)
	recommendations = recommend_movies(content_pred_df, user, games_data, user_item, 10)
	prod_rating = [(user,i,j) for i, j in zip(recommendations['product_id'].tolist(), recommendations['pred'].tolist())]
	results.extend(prod_rating)

# df_results = pd.DataFrame({'user_id': list(chainer(repeat(k, len(v)) for k, v in results.items())), 'product_ratings': list(chainer(results.values()))})
df_results = pd.DataFrame(results, columns=['user_id', 'product_id', 'rating'])
# df_results['product_ratings'].str[1:-1].str.split(',', expand=True).astype(float)

# output results
df_results.to_csv("cb5000.csv", index=False)


print(df_results.head(100))

# test recommendations for one user
recommendations = recommend_movies(content_pred_df, 76561198022842797, games_data, user_item, 10)
print(recommendations)





Total users: 11381
Total products: 7409


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print(ratings.head())
print(ratings.dtypes)

   level_0  index  Unnamed: 0  Unnamed: 0.1  ...  total_hours count  bin new_col
0        0      0           0            20  ...         11.0     1  1.5       1
1        1      5           5            73  ...          3.2     1  1.0       2
2        2     25          25           140  ...         18.4     1  1.0       3
3        3    634         642          2876  ...          7.3     1  1.0       4
4        4    658         666          3008  ...          0.5     1  1.0       5

[5 rows x 17 columns]
level_0             int64
index               int64
Unnamed: 0          int64
Unnamed: 0.1        int64
Unnamed: 0.1.1      int64
publisher          object
genres             object
title_x            object
tags               object
specs              object
product_id        float64
sentiment          object
user_id             int64
total_hours       float64
count               int64
bin               float64
new_col             int64
dtype: object


In [ ]:
# content-based recommender system

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from itertools import chain, repeat

chainer = chain.from_iterable

#---------------------- START functions ----------------------
# Function that takes in movie title as input and outputs most similar movies
# def get_recommendations(prod, cosine_sim):

#     # Get the pairwise similarity scores of games
#     sim_scores = list(enumerate(cosine_sim[prod]))

#     # Sort games based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 10 most similar games
#     sim_scores = sim_scores[1:11]

#     # Get the game indices
#     movie_indices = [i[0] for i in sim_scores]

#     # Return the top 10 most similar movies
#     return games_data.iloc[movie_indices]


def create_pivot_table(pd_df):
    data = pd_df.values
    rows, row_pos = np.unique(data[:, 4], return_inverse=True) # col 4: internal user id
    cols, col_pos = np.unique(data[:, 1], return_inverse=True) # col 1: real product id
    pivot_matrix = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_matrix[row_pos, col_pos] = data[:, 2]
    return pivot_matrix


def recommend_movies(pred_df, userID, games, original_ratings, num_recommendations):
    # get internal id of user
    uid_internal = map_uid.at[userID, 'index']

    sorted_user_predictions = pred_df.iloc[uid_internal].sort_values(ascending=False) 

    # get user's rated games
    user_data = original_ratings[original_ratings.user_id == int(userID)]
   	
   	# optional: merge game info
    #user_full = (user_data.merge(games, how='left', left_on='product_id', right_on='product_id').
    #                 sort_values(['rating'], ascending=False)
    #             )
    
    # Recommend the highest predicted rating games that the user hasn't seen yet.
    recommendations = games_data[~games_data['product_id'].isin(user_data['product_id'])].merge(pd.DataFrame(sorted_user_predictions).rename_axis('product_id').reset_index(), how='left', on='product_id').rename(columns={uid_internal: 'pred'}).sort_values('pred', ascending=False).iloc[:num_recommendations,:]

    return recommendations


#---------------------- END functions ----------------------

data = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/filter_for_algorithms.csv", usecols=['product_id', 'user_id', 'title_x', 'publisher', 'genres', 'tags', 'specs', 'bin'])
data['product_id'] = data['product_id'].astype(int) # product_id is initially a float with .0s
print("Total users: " + str(data['user_id'].nunique()))
print("Total products: " + str(data['product_id'].nunique()))

# dataframe storing games info
games_data = data[['product_id', 'title_x', 'publisher', 'genres']].copy()
games_data.fillna("", inplace=True)
games_data.drop_duplicates(inplace=True)
games_data.set_index(pd.Series(np.arange(len(games_data))), inplace=True)
games_data['metadata'] = games_data[['publisher', 'genres']].apply(lambda x: ' '.join(x), axis = 1)

# produce tfidf matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(games_data['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=games_data.index.tolist())

# compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#recommendations = get_recommendations(1, cosine_sim)
#print(recommendations)

# create internal product id
map_pid = games_data.reset_index()
map_pid = map_pid[['index', 'product_id']].copy()

# create internal user id
map_uid = data[['user_id']].copy().drop_duplicates(keep='first')
map_uid.set_index(pd.Series(np.arange(len(map_uid))), inplace=True)
map_uid.reset_index(inplace=True)
map_uid.set_index('user_id', inplace=True)

user_item = data[['user_id','product_id','bin']].copy()
user_item = user_item.merge(map_pid, on='product_id', how='left')
user_item = user_item.merge(map_uid, on='user_id', how='left')

#user_item_matrix = user_item[['user_id','index','bin']].pivot(index='user_id', columns='index', values = 'bin').fillna(0)
user_item_matrix2 = create_pivot_table(user_item)
pred = user_item_matrix2.dot(cosine_sim) / np.array([np.abs(cosine_sim).sum(axis=1)])
content_prediction = pred.clip(min=0)
content_pred_df = pd.DataFrame(content_prediction, columns=list(map_pid['product_id']))

# dict in the format {uid: [prod1, prod2, ..., prodN]}, stores all user's recommendations
results = {}

# iterate over all users
for index, user in enumerate(data['user_id'].unique().tolist()):
	if index%1000==0:
		print(index)
	recommendations = recommend_movies(content_pred_df, user, games_data, user_item, 10)
	results[user] = recommendations['product_id'].tolist()	


df_results = pd.DataFrame({'user_id': list(chainer(repeat(k, len(v)) for k, v in results.items())), 'product_id': list(chainer(results.values()))})

print(df_results.head(100))
df_results.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/content_based_result.csv", index = False)

# test recommendations for one user
recommendations = recommend_movies(content_pred_df, 76561198022842797, games_data, user_item, 10)
print(recommendations)





Total users: 11381
Total products: 7409


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: RuntimeWarning: invalid value encountered in true_divide


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
              user_id  product_id
0   76561198065664667      235900
1   76561198065664667      366510
2   76561198065664667      292230
3   76561198065664667      220700
4   76561198065664667      363890
..                ...         ...
95  76561198031545484      300400
96  76561198031545484        8600
97  76561198031545484        3050
98  76561198031545484        3010
99  76561198031545484      443860

[100 rows x 2 columns]


KeyError: ignored

Evaluation CB

In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/cb5000.csv")
ratings['split'] = np.random.randn(ratings.shape[0], 1)
msk = np.random.rand(len(ratings)) <= 0.8
train = ratings[msk]
test = ratings[~msk]
test.shape

(60989, 15)

In [ ]:
data['product_id'] = data['product_id'].astype('category')
merge = test.merge(data, on= ['user_id', 'product_id'], how='left')
merge.head()

merge['rating'] = merge['rating'].fillna(1)
merge.head()
user_pred = merge[['user_id', 'product_id', 'bin', 'rating']]
user_pred.rename(columns={'user_id': 'uid', 'rating': 'est', 'product_id': 'iid', 'bin':'r_ui'}, inplace=True)



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


def evaluation(predictions):
   
    res = {}
    
    #res['rmse'] = accuracy.rmse(predictions)
    y_true = user_pred['r_ui']
    y_pred = user_pred['est']
    res['rmse'] = mean_squared_error(y_true, y_pred, squared=False)
    
    res['mae'] = mean_absolute_error(y_true, y_pred)

    res['ndcg'] = ndcg(predictions)
    res['map'] = mean_ap(predictions)
    
    return res
print(evaluation(user_pred))

NDCG: 0.8180
MAP: 0.8870
{'rmse': 1.0894449315587942, 'mae': 0.46660053452261885, 'ndcg': 0.8180251436349244, 'map': 0.8869520743620831}
